In [2]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

#딥러닝 라이브러리
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN

tf.keras.backend.set_floatx('float64') #이 부분은 에러를 방지하기 위함임 중요코드 x

In [3]:
train = pd.read_csv('eletrain.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

60개의 건물, 총 85일, 24시간

In [4]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0,0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0,0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0,0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1,1
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1,1
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1,1
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1,1


60개 건물 7일씩 24시간인데 3시간마다 관측값 있음 + 비전기냉방설비운영, 태양광 보유 NaN

In [5]:
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,0,0
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,0,0
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,0,0
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,0,0
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,1,1
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,1,1
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1,1
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,1,1


LSTM 전력사용량만 변수로 사용

In [6]:
#전력사용량 정규화
mini = train.iloc[:, 2].min()
size = train.iloc[:,2].max() - train.iloc[:, 2].min()
train.iloc[:,2] = (train.iloc[:,2]-mini)/size

In [7]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,0.461072,17.6,2.5,92.0,0.8,0.0,0,0
1,1,2020-06-01 01,0.458624,17.7,2.9,91.0,0.3,0.0,0,0
2,1,2020-06-01 02,0.457017,17.5,3.2,91.0,0.0,0.0,0,0
3,1,2020-06-01 03,0.453729,17.1,3.2,91.0,0.0,0.0,0,0
4,1,2020-06-01 04,0.453437,17.0,3.3,92.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,0.231936,27.8,2.3,68.0,0.0,0.7,1,1
122396,60,2020-08-24 20,0.224119,27.3,1.2,71.0,0.0,0.0,1,1
122397,60,2020-08-24 21,0.201373,27.3,1.8,71.0,0.0,0.0,1,1
122398,60,2020-08-24 22,0.185982,27.1,1.8,74.0,0.0,0.0,1,1


In [8]:
input_window = 996 #임의의 수
output_window = 24 
window = 12 #window는 12시간 마다
num_features = 3 #나는 특성(전력, 기온, 습도) 사용
num_power = 60
end_ = 168
lstm_units = 64 #베이스라인은 32인데 난 64
dropout = 0.25 #베이스라인은 0.2 나는 25를 좋아해서 0.25
EPOCH = 50 #대충 40번은 돌려야할 것 같아서
BATCH_SIZE = 128

In [9]:
train_x = tf.reshape(train.iloc[:,[2,3,5]].values, [num_power, 24*85, num_features])

In [10]:
print(f'train_x.shape:{train_x.shape}')

train_x.shape:(60, 2040, 3)


In [11]:
train_window_x= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, input_window, num_features)) 
train_window_y= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, output_window, num_features))
print(f'train_window_x.shape:{train_window_x.shape}')
print(f'train_window_y.shape:{train_window_y.shape}')

train_window_x.shape:(60, 85, 996, 3)
train_window_y.shape:(60, 85, 24, 3)


In [15]:
train_x.shape[1]-(input_window + output_window)
#window를 왜 12시간으로 설정했을까?

NameError: ignored

In [16]:
for example in range(train_x.shape[0]):
  for start in range(0, train_x.shape[1]-(input_window + output_window), window):
    end = start+input_window
    train_window_x[example, start//window, :] = train_x[example, start: end, :]
    train_window_y[example, start//window, :] = train_x[example, end  : end + output_window, :]
  

train_window_x에 train 값 채우고
lstm에 넣기 위해 new_train_x를 만들고 reshape

In [17]:
new_train_x = tf.reshape(train_window_x, [-1, input_window, num_features])
new_train_y = tf.reshape(train_window_y, [-1, output_window, num_features])
print(f'new_train_x.shape:{new_train_x.shape}')
print(f'new_train_y.shape:{new_train_y.shape}')

new_train_x.shape:(5100, 996, 3)
new_train_y.shape:(5100, 24, 3)


여기서부터는 밑에서 여러 방법 해볼거임

In [18]:
#층 만들기
model = Sequential([LSTM(lstm_units, return_sequences=False, recurrent_dropout=dropout), Dense(output_window*num_features, kernel_initializer = tf.initializers.zeros()), Reshape([output_window, num_features])])

In [19]:
model.compile(optimizer = 'rmsprop', loss='mae', metrics=['mae'])

save_best_only = tf.keras.callbacks.ModelCheckpoint(filepath='lstm_model.h5', monitor='val_loss', save_best_only=True)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.1, patience=5)


In [20]:
history = model.fit(new_train_x, new_train_y, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split=0.2, verbose=0, callbacks=[early_stop, save_best_only, reduceLR])

KeyboardInterrupt: ignored

In [21]:
prediction = np.zeros((num_power, end_, num_features))
new_test_x = train_x

for i in range(end_//output_window):
  start_ = i*output_window
  next_ = model.predict(new_test_x[:, -input_window:, :])
  new_test_x = tf.concat([new_test_x, next_], axis=1)
  print(new_test_x.shape)
  prediction[:, start_:start_ + output_window,:] = next_
prediction = prediction*size + mini #다시 원래값 복구

2/2 [==============================] - 1s 508ms/step
(60, 2064, 3)
2/2 [==============================] - 1s 510ms/step
(60, 2088, 3)
2/2 [==============================] - 1s 507ms/step
(60, 2112, 3)
2/2 [==============================] - 1s 507ms/step
(60, 2136, 3)
2/2 [==============================] - 1s 511ms/step
(60, 2160, 3)
2/2 [==============================] - 1s 511ms/step
(60, 2184, 3)
2/2 [==============================] - 1s 521ms/step
(60, 2208, 3)


In [38]:
prediction[:,:,0]= prediction[:,:,0]*4
submission['answer'] = prediction[:,:,0].reshape([-1,1])
submission

,num_date_time,answer
0,1 2020-08-25 00,6824.315508
1,1 2020-08-25 01,7833.483282
2,1 2020-08-25 02,7093.779944
3,1 2020-08-25 03,7539.822094
4,1 2020-08-25 04,10516.218594
...,...,...
10075,60 2020-08-31 19,5451.253495
10076,60 2020-08-31 20,8480.692653
10077,60 2020-08-31 21,3822.792938
10078,60 2020-08-31 22,9763.331343


In [40]:
submission.to_csv('sample_submission.csv', index=False)